In this notebook, we wrap a TensorFlow model into a REST API using `BentoML`. The datasets used here are taken from [this](https://github.com/Nilabhra/kolkata_nlp_workshop_2019) repository. The notebook also takes references from [this](https://github.com/bentoml/BentoML/blob/master/examples/tf-keras-text-classification/tf-keras-text-classification.ipynb) example notebook from BentoML itself. 

In [1]:
import pandas as pd

train = pd.read_csv('https://raw.githubusercontent.com/Nilabhra/kolkata_nlp_workshop_2019/master/data/train.csv')
validation = pd.read_csv('https://raw.githubusercontent.com/Nilabhra/kolkata_nlp_workshop_2019/master/data/valid.csv')
test = pd.read_csv('https://raw.githubusercontent.com/Nilabhra/kolkata_nlp_workshop_2019/master/data/test.csv')

In [2]:
train.shape, validation.shape, test.shape

((9131, 3), (1142, 3), (1141, 3))

In [3]:
train.head()

,Unnamed: 0,text,class
0,3106,"I ordered a biryani, and the taste of the Biry...",positive
1,1927,A nice place to hangout since it has both the ...,positive
2,5405,This place is awesome for having lunch or dinn...,positive
3,7426,I got shell of egg in the egg roll. as a resul...,negative
4,1669,"Their biryani is oily, with a bit disconcertin...",negative


In [4]:
validation.head()

,Unnamed: 0,text,class
0,5373,The food was excellent with surplus quantity. ...,positive
1,7754,This place nearer to the Gitanjali metro stati...,positive
2,11134,Ordered for Aloo tikki with choley just now @0...,negative
3,365,Hatari is one of those restaurants that our fa...,positive
4,8021,Disappointing.......\nThey have altered the ta...,negative


In [5]:
test.head()

,Unnamed: 0,text,class
0,24,This place is amazing. I think the best place ...,positive
1,25,This place has been on my list for quite some ...,positive
2,37,What a wonderful cold winter evening it was. M...,positive
3,49,BabBQ had always been a personal favorite when...,positive
4,60,Know for its Deep Dish Pizza this place is sur...,negative


In [6]:
train['text'].loc[0]

'I ordered a biryani, and the taste of the Biryani was beyond my expectations and the quantity was also enough comparatively to the price!\nReally nice much appreciable'

### Removing digits for the text

In [7]:
from string import digits

def remove_digits(s):
    remove_digits = str.maketrans('', '', digits)
    res = s.translate(remove_digits)
    return res

In [8]:
train['text'] = train['text'].apply(remove_digits)
validation['text'] = validation['text'].apply(remove_digits)

### Bag of words representation 

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words=None, lowercase=True,
                             ngram_range=(1, 1), min_df=2, binary=True)

train_features = vectorizer.fit_transform(train['text'])
train_labels = train['class']

valid_features = vectorizer.transform(validation['text'])
valid_labels = validation['class']

## Label encode the classes

In [10]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train_labels = le.fit_transform(train_labels)
valid_labels = le.transform(valid_labels)

### Model building and compilation

In [11]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dropout, Dense

In [12]:
model = keras.Sequential()

model.add(Dropout(rate=0.2, input_shape=train_features.shape[1:]))
for _ in range(2):
        model.add(Dense(units=64, activation='relu'))
        model.add(Dropout(rate=0.2))
model.add(Dense(units=1, activation='sigmoid'))

In [13]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])

In [14]:
# Define an EarlyStopping callback
es_cb = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

### We are ready to train the model and validate

In [15]:
model.fit(train_features,
                    train_labels,
                    epochs=15,
                    batch_size=512,
                    validation_data=(valid_features, valid_labels),
                    callbacks=[es_cb],
                    verbose=1)

Train on 9131 samples, validate on 1142 samples
Epoch 1/15
9131/9131 [==============================] - 1s 130us/sample - loss: 0.6429 - acc: 0.6284 - val_loss: 0.5817 - val_acc: 0.6524
Epoch 2/15
9131/9131 [==============================] - 1s 78us/sample - loss: 0.5335 - acc: 0.7363 - val_loss: 0.4670 - val_acc: 0.8161
Epoch 3/15
9131/9131 [==============================] - 1s 78us/sample - loss: 0.4249 - acc: 0.8192 - val_loss: 0.4091 - val_acc: 0.8327
Epoch 4/15
9131/9131 [==============================] - 1s 79us/sample - loss: 0.3561 - acc: 0.8519 - val_loss: 0.4035 - val_acc: 0.8336
Epoch 5/15
9131/9131 [==============================] - 1s 80us/sample - loss: 0.3145 - acc: 0.8706 - val_loss: 0.4087 - val_acc: 0.8257
Epoch 6/15
9131/9131 [==============================] - 1s 79us/sample - loss: 0.2690 - acc: 0.8939 - val_loss: 0.4183 - val_acc: 0.8257
Epoch 7/15
9131/9131 [==============================] - 1s 80us/sample - loss: 0.2421 - acc: 0.9029 - val_loss: 0.4397 - val_acc:

### How good is the model? 

In [16]:
test['text'] = test['text'].apply(remove_digits)
test_features = vectorizer.transform(test['text'])
test_labels = le.transform(test['class'])

In [17]:
results = model.evaluate(test_features, test_labels)
print("Accuracy: {0:.2f}%".format(results[1]*100.))

1141/1141 [==============================] - 0s 59us/sample - loss: 0.5749 - acc: 0.7993
Accuracy: 79.93%


### Combining the training and validation sets and retraining the model

In [18]:
data = pd.concat((train, validation), axis=0)

vectorizer = CountVectorizer(stop_words=None, lowercase=True,
                             ngram_range=(1, 1), min_df=2)

features = vectorizer.fit_transform(data['text'])
labels = le.fit_transform(data['class'])

test_features = vectorizer.transform(test['text'])
test_labels = le.transform(test['class'])

In [19]:
model = keras.Sequential()

model.add(Dropout(rate=0.2, input_shape=features.shape[1:]))
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(rate=0.2))
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(rate=0.2))
model.add(Dense(units=1, activation='sigmoid'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])

In [20]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])

In [21]:
model.fit(features,
                    labels,
                    epochs=15,
                    batch_size=512,
                    validation_data=(test_features, test_labels),
                    callbacks=[es_cb],
                    verbose=1)

Train on 10273 samples, validate on 1141 samples
Epoch 1/15
10273/10273 [==============================] - 1s 120us/sample - loss: 0.6191 - acc: 0.6436 - val_loss: 0.5313 - val_acc: 0.7432
Epoch 2/15
10273/10273 [==============================] - 1s 90us/sample - loss: 0.4823 - acc: 0.7826 - val_loss: 0.4332 - val_acc: 0.8186
Epoch 3/15
10273/10273 [==============================] - 1s 93us/sample - loss: 0.3915 - acc: 0.8322 - val_loss: 0.4259 - val_acc: 0.8300
Epoch 4/15
10273/10273 [==============================] - 1s 91us/sample - loss: 0.3289 - acc: 0.8663 - val_loss: 0.4431 - val_acc: 0.8300
Epoch 5/15
10273/10273 [==============================] - 1s 92us/sample - loss: 0.2928 - acc: 0.8806 - val_loss: 0.4491 - val_acc: 0.8177
Epoch 6/15
10273/10273 [==============================] - 1s 92us/sample - loss: 0.2507 - acc: 0.8991 - val_loss: 0.4895 - val_acc: 0.8046
Epoch 7/15
10273/10273 [==============================] - 1s 91us/sample - loss: 0.2258 - acc: 0.9134 - val_loss: 0.

> We will use this model for serving. 

### Inferencing on a single test sample

In [22]:
test_sample = vectorizer.transform([remove_digits('I had a very bad experience you know.')])
le.inverse_transform(model.predict_classes(test_sample))

/Users/rakeshkumar/miniconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array([['negative']], dtype=object)

### Model saving and serving just got easier

In [23]:
%%writefile text_classification_service.py
import pandas as pd
from tensorflow import keras
import tensorflow as tf
from string import digits
from bentoml import api, env, BentoService, artifacts
from bentoml.artifact import TfKerasModelArtifact, PickleArtifact
from bentoml.handlers import JsonHandler

@artifacts([
  TfKerasModelArtifact('model'),
  PickleArtifact('vectorizer')
])
@env(conda_dependencies=['tensorflow', 'scikit-learn'])
class TextClassificationService(BentoService):
    
    @api(JsonHandler)
    def predict(self, parsed_json):
        text = parsed_json['text']
        remove_digits = str.maketrans('', '', digits)
        text = text.translate(remove_digits)
        text = self.artifacts.vectorizer.transform([text])
        prediction =  self.artifacts.model.predict_classes(text)[0][0]
        if prediction==0:
            response = {'Sentiment': 'Negative'}
        elif prediction==1:
            response = {'Sentiment': 'Positive'}
        
        return response

Overwriting text_classification_service.py


In [24]:
from text_classification_service import TextClassificationService

# Construct the vectorizer once again for the artifact
vectorizer = CountVectorizer(stop_words=None, lowercase=True,
                             ngram_range=(1, 1), min_df=2)

train = pd.read_csv('https://raw.githubusercontent.com/Nilabhra/kolkata_nlp_workshop_2019/master/data/train.csv')
validation = pd.read_csv('https://raw.githubusercontent.com/Nilabhra/kolkata_nlp_workshop_2019/master/data/valid.csv')

def remove_digits(s):
    remove_digits = str.maketrans('', '', digits)
    res = s.translate(remove_digits)
    return res

train['text'] = train['text'].apply(remove_digits)
validation['text'] = validation['text'].apply(remove_digits)

data = pd.concat((train, validation), axis=0)

vectorizer.fit_transform(data['text'])

features = vectorizer.fit_transform(data['text'])

# Save and serve
svc = TextClassificationService.pack(model=model, vectorizer=vectorizer)
saved_path = svc.save('./text_classification')
print(saved_path)
!ls {saved_path}

./text_classification/TextClassificationService/2019_06_26_fad1818a
Dockerfile                TextClassificationService requirements.txt
MANIFEST.in               bentoml.yml               setup.py
README.md                 environment.yml


When you execute - 
`bentoml serve ./text_classification/TextClassificationService/0.0.2019_04_22_e5d9f7d0/` and hit Postman with the following configuration - 
![](https://i.ibb.co/C70GzQg/Screen-Shot-2019-04-22-at-8-40-27-AM.png)
you should get the desired result. 

In [25]:
svc.predict({"text": "I had a wonderful experience eating their chicken noodles! Also loved the ambience."})

{'Sentiment': 'Positive'}

> Correct! 

In [26]:
!rm -R {saved_path}